In [ ]:
import os 
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())
openai_api_key = os.getenv('OPENAI_API_KEY')

In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [3]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser
chain.invoke({"soccer_player": "Lionel Messi"})

'A curious fact about Lionel Messi is that he was diagnosed with a growth hormone deficiency as a child, which stunted his growth. This condition required expensive medical treatments that his family could not afford, but Barcelona FC offered to pay for his treatments in exchange for Messi joining their youth academy. This decision turned out to be a wise investment for the club, as Messi went on to become one of the greatest footballers of all time.'

## Use of .bind() to add arguments to a Runnable in a LCEL Chain

For example, we can add an arguement to stop the model response when it reaches a word "Ronaldo".
"""

In [6]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser
chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano '

## Combining LCEL Chains

In [7]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

chain = prompt | model | output_parser

In [9]:
chain.invoke({"soccer_player": "Ronlado"})

'One curious fact about Cristiano Ronaldo is that he is known for his incredible work ethic and commitment to physical fitness. He reportedly has the physical capacity of a 20-year-old despite being in his mid-30s, thanks to his strict training regimen and dedication to taking care of his body. This has allowed him to maintain his high level of performance and competitiveness throughout his career.'

## Combined Chain

In [10]:
historian_prompt = ChatPromptTemplate.from_template("Was {soccer_player} most earning player in 2021?")
compose_chain = {"soccer_player": chain, "fact": chain} | historian_prompt | model | output_parser
compose_chain.invoke({"soccer_player": "Ronaldo"})

"One curious fact about Cristiano Ronaldo is that he has his own museum dedicated to his life and career in his hometown of Funchal, Madeira. The museum showcases memorabilia, trophies, and personal items related to Ronaldo's success as a professional soccer player. Ronaldo was also the highest-earning football player in 2021, according to Forbes."

## Another Example a Chain Inside another chain

In [15]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Define prompt templates
prompt1 = ChatPromptTemplate.from_template("What is the country of {soccer_player}?")
prompt2 = ChatPromptTemplate.from_template("What continent is {country} in? Respond in {language}.")

# Initialize model
model = ChatOpenAI()

# Define first chain
chain1 = prompt1 | model | StrOutputParser()

# Define second chain correctly
chain2 = {
    "country": chain1,
    "language": itemgetter("language")
} | prompt2 | model | StrOutputParser()

# Test the chain
response = chain2.invoke({"soccer_player": "Ronaldo", "language": "Portuguese"})
print(response)


Cristiano Ronaldo é da Europa. Ele é de Portugal.


## Rag Demo Application

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = {
    "context": retriever | format_docs,
    "question": RunnablePassthrough()
    | prompt
    | model
    | StrOutputParser()
}


In [ ]:
rag_chain.invoke({"question": "What is the agent-based model?"})